In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
import cv2
from skimage.morphology import medial_axis
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import FileUpload
from io import BytesIO
from pathlib import Path
from IPython.display import display 
# pip install ipywidgets

# Display the upload widget
display(file_upload)



In [ ]:
# --- Masking Function ---
def create_mask(image, threshold=150):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    return mask


In [ ]:
def lava_width(PIV_masked, quantile=1.0, thresh=0.2, min_dist=2):
    mask = PIV_masked.where(PIV_masked == 0).fillna(1)
    PIV_masked['mask'] = mask

    pixX = (PIV_masked.x[1] - PIV_masked.x[0]).values
    pixY = (PIV_masked.y[1] - PIV_masked.y[0]).values

    skel, distance = medial_axis(mask, return_distance=True)

    dist_on_skeleton = distance * skel * 2 * pixX
    PIV_masked['centerline'] = xr.DataArray(dist_on_skeleton, dims=['x', 'y'])

    # Display Results
    fig, ax = plt.subplots(figsize=(8, 8))
    PIV_masked.plot(ax=ax, cmap='magma')
    PIV_masked.mask.plot.contour(levels=1, yincrease=False, cmap='white', ax=ax)

    print('Mean width:', round(np.mean(dist_on_skeleton[dist_on_skeleton > 0]), 1), 'm')
    print('Max width:', round(np.max(dist_on_skeleton), 1), 'm')
    print('Min width:', round(np.min(dist_on_skeleton[dist_on_skeleton > 0]), 1), 'm')


In [ ]:
def process_video(video_path, threshold=150):
    cap = cv2.VideoCapture(video_path)
    
    flow_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
        
        flow_data.append(mask.astype(float) / 255)
        
        cv2.imshow('Lava Flow Frame', mask)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    avg_flow_data = np.mean(flow_data, axis=0)
    
    plt.figure(figsize=(8, 4))
    plt.imshow(avg_flow_data, cmap=cm.magma, vmin=0, vmax=1)
    plt.title("Lava Flow Visualization (Averaged Over Time)")
    plt.colorbar(label='Flow Intensity')
    plt.show()


In [ ]:
file_upload = FileUpload(accept='.mp4', multiple=False)
file_upload

def handle_upload(change):
    file_info = list(change['new'].values())[0]
    file_path = Path("uploaded_video.mp4")
    with open(file_path, 'wb') as f:
        f.write(file_info['content'])

    print(f"File saved as: {file_path}")
    process_video(str(file_path))  # Run analysis automatically

file_upload.observe(handle_upload, names='value')
